In [ ]:
!make clean

# Modern C++ part II: more than templates

1. Copy elision / return value optimization
2. Move semantics and copy elision
    1. Forced move is a bad idea
3. Data concatenation
    1. Style 1: return `vector`
    2. Style 2: use output `vector`
    3. Style 3: use a class for both return and output argument
4. Variadic template
5. Perfect forwarding
6. Lambda expression
    1. Keep a lambda in a local variable
    2. Difference between `auto` and `std::function`
7. Closure
    1. Comments on functional style

# Copy elision / return value optimization

Copy elision is one of the two forms of optimization, alongside allocation elision and extension, that is allowed to change the side effects.

Sometimes copy elision is also called return value optimization (RVO) or named return value optimization (NRVO).

In the following example (`03_elision/01_copy.cpp`), even though no optimization flag is used, the copy is optimized out:

```cpp
class IsCopied
{
public:
    static IsCopied & instance()
    {
        static IsCopied inst;
        return inst;
    }

    IsCopied & void on() { m_status = true; return *this; }
    operator bool() const { return m_status; }

private:
    IsCopied() : m_status(false) {}
    bool m_status;
};

class Data
{
public:
    constexpr const static size_t NELEM = 1024*8;
    Data()
    {
        std::cout << "Data constructed @" << this << std::endl;
    }
    Data(Data const & other)
    {
        copy_from(other);
        std::cout << "Data copied to @" << this << " from @" << &other << std::endl;
    }
    ~Data()
    {
        std::cout << "Data destructed @" << this << std::endl;
    }
    void copy_from(Data const & other)
    {
        for (size_t it=0; it < NELEM; ++it)
        {
            m_buffer[it] = other.m_buffer[it];
        }
        IsCopied::instance().on();
    }
};

void manipulate_with_reference(Data & data, int value)
{
    std::cout << "Manipulate with reference: " << &data << std::endl;

    for (size_t it=0; it < data.size(); ++it)
    {
        data[it] = value + it;
    }
    // In a real consumer function we will do much more meaningful operations.

    // However, we cannot destruct an object passed in with a reference.
}

Data worker1()
{
    Data data;

    // Manipulate the Data object.
    manipulate_with_reference(data, 3);

    return data;
}

Data worker2()
{
    Data data = worker1();

    // Manipulate the Data object, again.
    manipulate_with_reference(data, 8);

    return data;
}

int main(int argc, char ** argv)
{
    std::cout
        << (bool(IsCopied::instance()) ? "Something" : "Nothing")
        << " is copied" << std::endl;
    Data data = worker2();
    std::cout
        << (bool(IsCopied::instance()) ? "Something" : "Nothing")
        << " is copied" << std::endl;
}
```

In [ ]:
!make -C 03_elision clean ; make -C 03_elision OPT= 01_copy
#!r2 -Aqc "e scr.color=0 ; afl" 03_elision/01_copy
!r2 -Aqc "e scr.color=0 ; s sym.worker1 ; pdf ; s sym.worker2 ; pdf" 03_elision/01_copy

In [ ]:
!make -C 03_elision clean ; make -C 03_elision OPT= 01_copy
!03_elision/01_copy

# Move semantics and copy elision

Move semantics greatly helps us to avoid copying expensive resources.  To take advantage of that, our `Data` class should be changed to use dynamic allocation (`03_elision/02_move.cpp`):

```cpp
class Data
{

public:

    constexpr const static size_t NELEM = 1024*8;

    Data()
    {
        m_buffer = new int[NELEM];
        std::cout << "Data constructed @" << this << std::endl;
    }

    Data(Data const & other)
    {
        m_buffer = new int[NELEM];
        copy_from(other);
        std::cout << "Data copied to @" << this << " from @" << &other << std::endl;
    }

    Data & operator=(Data const & other)
    {
        if (nullptr == m_buffer) { m_buffer = new int[NELEM]; }
        copy_from(other);
        std::cout << "Data copy assigned to @" << this << " from @" << &other << std::endl;
        return *this;
    }

    Data(Data && other)
    {
        m_buffer = other.m_buffer;
        other.m_buffer = nullptr;
        std::cout << "Data moved to @" << this << " from @" << &other << std::endl;
        Status::instance().set_moved();
    }

    Data & operator=(Data && other)
    {
        if (m_buffer) { delete[] m_buffer; }
        m_buffer = other.m_buffer;
        other.m_buffer = nullptr;
        std::cout << "Data move assigned to @" << this << " from @" << &other << std::endl;
        Status::instance().set_moved();
        return *this;
    }

    ~Data()
    {
        if (m_buffer) { delete[] m_buffer; }
        std::cout << "Data destructed @" << this << std::endl;
    }

```

## Forced move is a bad idea

Although the move semantics indeed avoids copy the expensive buffer in the `Data` class, it cannot avoid copy the `Data` object itself.  However, copy elision (RVO & NRVO) can avoid copy the `Data` object.

```cpp
Data worker1()
{
    Data data;

    // Manipulate the Data object.
    manipulate_with_reference(data, 3);

    return data;
}

Data worker2()
{
    Data data = worker1();

    // Manipulate the Data object, again.
    manipulate_with_reference(data, 8);

#ifdef FORCEMOVE
    // Implicit move semantics destroys copy elision.
    return std::move(data);
#else
    return data;
#endif
}

int main(int argc, char ** argv)
{
    std::cout
        << "Status:"
        << (bool(Status::instance().is_copied()) ? " copied" : " uncopied")
        << (bool(Status::instance().is_moved()) ? " moved" : " unmoved")
        << std::endl;
    Data data = worker2();
    std::cout
        << "Status:"
        << (bool(Status::instance().is_copied()) ? " copied" : " uncopied")
        << (bool(Status::instance().is_moved()) ? " moved" : " unmoved")
        << std::endl;
}
```

In [ ]:
# See what happens with forced move.
!make -C 03_elision clean ; make -C 03_elision FLAGS=-DFORCEMOVE OPT= 02_move
!03_elision/02_move

In [ ]:
# Let compiler perform copy elision / NRVO.
!make -C 03_elision clean ; make -C 03_elision OPT= 02_move
!03_elision/02_move

# Data concatenation

Because of copy elision, for readibility in C++ it is prefer to write:

```cpp
std::vector<int> worker_return();
```

than

```cpp
void worker_argument(std::vector<int> & output /* output argument */);
```

Because in consumer code:

```cpp
// It reads clearly that the worker produces new result.
std::vector<int> result = worker_return();

// It takes a second to understand that the worker is using result as a buffer
// for output.
std::vector<int> result;
worker_argument(result);

/*
 * The result is pre-populated before sending to the worker.  From the
 * following lines we can't know how the worker will use result.
 *
 * By reading the worker signature we know that result may be used for output.
 * We can only be sure that result is used for output after reading the full
 * implemnetation of the worker.
 *
 * The worker may or may not expect the output argument to be pre-populated.
 * Regardless, it has to use runtime check to ensure either case.
 */
std::vector<int> result(100);
std::fill(result.begin(), result.end(), 7);
worker_argument(result);
```

The ambiguity is a productivity killer.  (Runtime performance is another story.)

## Style 1: return `vector`

The first style returns a vector from inner and appends it in outer.  It is easier to read and test.  The inner worker:

```cpp
std::vector<Data> inner1(size_t start, size_t len)
{
    std::cout << "** inner1 begins with " << start << std::endl;
    std::vector<Data> ret;
    for (size_t it=0; it < len; ++it)
    {
        Data data(start+it);
        ret.emplace_back(std::move(data));
    }
    return ret;
}
```

The outer worker:

```cpp
void outer1(size_t len)
{
    std::cout << "* outer1 begins" << std::endl;
    std::vector<Data> vec;
    for (size_t it=0; it < len; ++it)
    {
        std::cout << std::endl;
        std::cout << "* outer1 loop it=" << it << " begins" << std::endl;
        std::vector<Data> subvec = inner1(vec.size(), it+1);
        std::cout << "* outer1 obtained inner1 at " << vec.size() << std::endl;
        vec.insert(
            vec.end()
          , std::make_move_iterator(subvec.begin())
          , std::make_move_iterator(subvec.end())
        );
        std::cout << "* outer1 inserted subvec.size()=" << subvec.size() << std::endl;
    }
    std::cout << "* outer1 result.size() = " << vec.size() << std::endl << std::endl;
}
```

In [ ]:
# Style 1 result
!make -C 03_elision clean ; make -C 03_elision FLAGS="-DOTYPE=1" 03_accumulate
!03_elision/03_accumulate

The unwanted copies come from `std::vector` resizing.  To mitigate it, we should mark the move constructor with `noexcept`:

```cpp
Data(Data && other) noexcept
{
    m_serial = other.m_serial;
    m_buffer = other.m_buffer;
    other.m_buffer = nullptr;
    std::cout << "Data #" << m_serial << " moved to @" << this << " from @" << &other << std::endl;
}
```

In [ ]:
# Unwanted copies are turned into move.
!make -C 03_elision clean ; make -C 03_elision FLAGS="-DMOVENOEXCEPT -DOTYPE=1" 03_accumulate
!03_elision/03_accumulate

## Style 2: use output `vector`

The second style uses an output argument which is passed from outer to inner.  The inner worker:

```cpp
void inner2(size_t start, size_t len, std::vector<Data> & result /* for output */)
{
    std::cout << "** inner2 begins with " << start << std::endl;
    for (size_t it=0; it < len; ++it)
    {
        Data data(start+it);
        result.emplace_back(std::move(data));
    }
}
```

The outer worker:

```cpp
void outer2(size_t len)
{
    std::cout << "* outer2 begins" << std::endl;
    std::vector<Data> vec;
    for (size_t it=0; it < len; ++it)
    {
        std::cout << std::endl;
        std::cout << "* outer2 loop it=" << it << " begins" << std::endl;
        inner2(vec.size(), it+1, vec);
    }
    std::cout << "* outer2 result.size() = " << vec.size() << std::endl << std::endl;
}
```

There is no longer the intermediate vector and it saves quite a number of movement.  The prize we pay is less testability.

In [ ]:
# The output argument results into less movement.
!make -C 03_elision clean ; make -C 03_elision FLAGS="-DMOVENOEXCEPT -DOTYPE=2" 03_accumulate
!03_elision/03_accumulate

## Style 3: use a class for both return and output argument

The third style uses a class so that it support both vector returnning and output argument for the vector.  The class is:

```cpp
struct Accumulator
{

public:
    // This can be called if consumers want the sub-operation one by one, and
    // make the code more testable. But it isn't really used in the example.
    std::vector<Data> inner1(size_t start, size_t len)
    {
        std::cout << "** Accumulator::inner1 begins with " << start << std::endl;
        std::vector<Data> ret;
        ret.reserve(len);
        inner2(start, len, ret);
        return ret;
    }

private:
    void inner2(size_t start, size_t len, std::vector<Data> & ret)
    {
        std::cout << "** Accumulator::inner2 begins with " << start << std::endl;
        for (size_t it=0; it < len; ++it)
        {
            Data data(start+it);
            ret.emplace_back(std::move(data));
        }
    }

public:
    // This is used when batch operation is in demand.
    void outer(size_t len)
    {
        std::cout << "* Accumulator::outer begins" << std::endl;
        result.reserve(len*(len+1)/2);
        for (size_t it=0; it < len; ++it)
        {
            std::cout << std::endl;
            std::cout << "* Accumulator::outer loop it=" << it << " begins" << std::endl;
            inner2(result.size(), it+1, result);
        }
        std::cout << "* Accumulator::outer result.size() = " << result.size() << std::endl << std::endl;
    }

public:
    std::vector<Data> result;

}; /* end struct Accumulator */
```

Although `Accumulator::outer` still calls the function `Accumulator::inner2` that takes an output argument, we also have the function `Accumulator::inner1` that wraps around `Accumulator::inner2` and make it testable.

To further save unwanted movements, we pre-calculate the number of elements to be populated in the vector and reserve the space.

In [ ]:
# The class-style implementation is also enhanced with reserve.
!make -C 03_elision clean ; make -C 03_elision FLAGS="-DMOVENOEXCEPT -DOTYPE=3" 03_accumulate
!03_elision/03_accumulate

Evolution of the three styles demonstrate how one may develop sophisticated code from a standalone helper to an optimized class library.

# Variadic template

Variadic template allows us to capture any number of template arguments in a function template.  Assuming we have 2 constructors for `Data`:

```cpp
Data(size_t serial, ctor_passkey const &)
  : m_serial(serial)
{
    m_buffer = new int[NELEM];
    initialize(0);
    std::cout << "Data #" << m_serial << " constructed @" << this
              << "(serial=" << m_serial << ")" << std::endl;
}

Data(size_t serial, int base, ctor_passkey const &)
  : m_serial(serial+base)
{
    m_buffer = new int[NELEM];
    initialize(0);
    std::cout << "Data #" << m_serial << " constructed @" << this
              << "(serial=" << m_serial << ")"
              << "(base=" << base << ")" << std::endl;
}
```

We will need two factories methods for them:

```cpp
static std::shared_ptr<Data> create(size_t serial)
{
    return std::make_shared<Data>(serial, ctor_passkey());
}

static std::shared_ptr<Data> create(size_t serial, int base)
{
    return std::make_shared<Data>(serial, int, ctor_passkey());
}
```

It's tedious to add the corresponding factory functions, although it is not too much an issue, since the compiler will complain.  Let's assume we forgot the add the second factory overload and see what may happen.

In [ ]:
!make -C 04_template clean ; make -C 04_template FLAGS=-DUSE_CREATE 01_factory

Variadic template can conveniently help us summarize the two overloads into one template function, and also capture every new public constructor that will be added in the future.

```cpp
template < typename ... Args >
static std::shared_ptr<Data> make(Args && ... args)
{
    // Forget about the 'forward' for now. It will be discussed later.
    return std::make_shared<Data>(std::forward<Args>(args) ..., ctor_passkey());
}
```

Run the following code:

```cpp
void outer1(size_t len)
{
    std::cout << "* outer1 begins" << std::endl;
    std::vector<std::shared_ptr<Data>> vec;
    for (size_t it=0; it < len; ++it)
    {
        std::cout << std::endl;
        std::cout << "* outer1 loop it=" << it << " begins" << std::endl;
        std::vector<std::shared_ptr<Data>> subvec = inner1(vec.size(), it+1);
        std::cout << "* outer1 obtained inner1 at " << vec.size() << std::endl;
        vec.insert(
            vec.end()
          , std::make_move_iterator(subvec.begin())
          , std::make_move_iterator(subvec.end())
        );
        std::cout << "* outer1 inserted subvec.size()=" << subvec.size() << std::endl;
    }
    std::cout << "* outer1 result.size() = " << vec.size() << std::endl << std::endl;

    std::cout << "* outer1 end" << std::endl << std::endl;
}

std::vector<std::shared_ptr<Data>> inner1(size_t base, size_t len)
{
    std::cout << "** inner1 begins with " << base << std::endl;
    std::vector<std::shared_ptr<Data>> ret;
    for (size_t it=0; it < len; ++it)
    {
        std::shared_ptr<Data> data;
        if (0 == base)
        {
            data = Data::make(it);
        }
        else
        {
            data = Data::make(it, base);
        }
        ret.emplace_back(data);
    }
    return ret;
}
```

In [ ]:
!make -C 04_template clean ; make -C 04_template 01_factory
!04_template/01_factory

# Perfect forwarding

In the previous section we used `std::forward`, which enables perfect forwarding:

```cpp
template < typename ... Args >
static std::shared_ptr<Data> make(Args && ... args)
{
    return std::make_shared<Data>(std::forward<Args>(args) ..., ctor_passkey());
}, 
```

Although the template is named `forward`, it doesn't forward anything.  Like `std::move`, it serves as a cast to rvalue reference.  The difference is that:

1. `std::move` unconditionally casts the input to rvalue reference.
2. `std::forward` casts to rvalue reference only when it can.

When we write `Data &&`, it is a rvalue reference.  With `T &&` as a template argument, when we write `T &&`, it can be either lvalue or rvalue, so it is also called universal reference.  The rule of thumb is that when `T` is a deductible type (`auto &&` falls into this category too), `T &&` is a universal reference rather a strict rvalue reference.

So `std::forward<Args>(args)` preserves the type of reference of the arguments, and the pattern is called perfect forwarding.  Because the arguments of `Data` constructors were both fundamental types, it doesn't matter whether or not we use perfect forwarding.  To demonstrate how it works, we add the two wrapper:

```cpp
// Proxy to copy and move constructor.
Data(Data const &  other, ctor_passkey const &) : Data(std::forward<Data const &>(other)) {}
Data(Data       && other, ctor_passkey const &) : Data(std::forward<Data &&>(other)) {}
```

And we use a slightly different `outer`:

```cpp
void outer1(size_t len)
{
    std::cout << "* outer1 begins" << std::endl;
    std::vector<std::shared_ptr<Data>> vec;
    for (size_t it=0; it < len; ++it)
    {
        std::cout << std::endl;
        std::cout << "* outer1 loop it=" << it << " begins" << std::endl;
        std::vector<std::shared_ptr<Data>> subvec = inner1(vec.size(), it+1);
        std::cout << "* outer1 obtained inner1 at " << vec.size() << std::endl;
        vec.insert(
            vec.end()
          , std::make_move_iterator(subvec.begin())
          , std::make_move_iterator(subvec.end())
        );
        std::cout << "* outer1 inserted subvec.size()=" << subvec.size() << std::endl;
    }
    std::cout << "* outer1 result.size() = " << vec.size() << std::endl << std::endl;

    // Exercise the perfect forwarding.
    vec.emplace_back(Data::make(*vec[0]));
    vec.emplace_back(Data::make(std::move(*vec[1])));

    std::cout << "* outer1 end" << std::endl << std::endl;
}
```

In [ ]:
# Perfect forwarding dispatches to the correct constructors.
!make -C 04_template clean ; make -C 04_template FLAGS=-DSHOW_PERFECT_FORWARD 01_factory
!04_template/01_factory

# Lambda expression

C++ lambda expression enables a shorthand for anonymous function.  The syntax (no variable is captured) is:

```cpp
[] (/* arguments */) { /* body */ }
```

It works basically like a functor.

```cpp
struct Functor
{
    bool operator()(int v)
    {
        return 0 == v % 23;
    }
}; /* end struct Functor */

int main(int argc, char ** argv)
{
    std::vector<int> data(63712);
    for (size_t i=0 ; i<data.size(); ++i) { data[i] = i;}

    std::cout
        << "Number divisible by 23 (count by functor): "
        << std::count_if(data.begin(), data.end(), Functor())
        << std::endl;

    std::cout
        << "Number divisible by 23 (count by lambda): "
        << std::count_if(data.begin(), data.end(), [](int v){ return 0 == v%23; })
        << std::endl;

    return 0;
}
```

In [ ]:
!make -C 05_lambda clean ; make -C 05_lambda 01_lambda
!05_lambda/01_lambda

## Keep a lambda in a local variable

Lambda is considered as anonymous function, but we can give it a 'name' by assigning it to a variable.  There are two choices: `auto` or `std::function`.

```cpp
int main(int argc, char ** argv)
{
    std::vector<int> data(63712);
    for (size_t i=0 ; i<data.size(); ++i) { data[i] = i;}

    std::cout
        << "Number divisible by 23 (count by lambda inline): "
        << std::count_if(data.begin(), data.end(), [](int v){ return 0 == v%23; })
        << std::endl;

    auto condition = [](int v){ return 0 == v%23; };

    std::cout
        << "Number divisible by 23 (count by lambda in auto): "
        << std::count_if(data.begin(), data.end(), condition)
        << std::endl;

    std::function<bool (int)> condition_function = [](int v){ return 0 == v%23; };

    std::cout
        << "Number divisible by 23 (count by lambda in std::function): "
        << std::count_if(data.begin(), data.end(), condition_function)
        << std::endl;

    return 0;
}
```

In [ ]:
!make -C 05_lambda clean ; make -C 05_lambda 02_stored
!05_lambda/02_stored

## Difference between `auto` and `std::function`

Although both `auto` and `std::function` can hold a lambda, the two ways are not exactly the same.  A lambda works like a functor and the `auto` type reflects that.  A `std::function` is more versatile than it, and takes more memory as well.

This is a list of targets (callables) that a `std::function` can hold: free functions, member functions, functors, lambda expressions, and bind expressions.

```cpp
std::cout
    << std::endl
    << "The differences between lambda and std::function"
    << std::endl;
std::cout
    << "type name of lambda: "
    << typeid(condition).name() << std::endl;
std::cout
    << "type name of std::function: "
    << typeid(condition_function).name() << std::endl;

std::cout
    << "size of lambda: "
    << sizeof(condition) << std::endl;
std::cout
    << "size of std::function: "
    << sizeof(condition_function) << std::endl;
```

In [ ]:
!make -C 05_lambda clean ; make -C 05_lambda FLAGS=-DSHOW_DIFF 02_stored
!05_lambda/02_stored

# Closure

So far our use of lambda expressions doesn't capture any local variables.  When it does, we call the lambda expression a closure.

We must tell the compiler what type of capture the lambda espression would like to use.  Otherwise the compilation fails.

```cpp
int main(int argc, char ** argv)
{
    std::vector<int> data(63712);
    for (size_t i=0 ; i<data.size(); ++i) { data[i] = i;}

    int divisor = 23;

#if WRONG_CAPTURE
    std::cout
        << "Count (wrong capture): "
        << std::count_if(data.begin(), data.end(), [](int v){ return 0 == v%divisor; })
        << " (divisor: " << divisor << ")"
        << std::endl;
#endif

    return 0;
}
```

In [ ]:
!make -C 05_lambda clean ; make -C 05_lambda FLAGS=-DWRONG_CAPTURE 03_closure

We may explicitly tell the compiler that we want `divisor` to be captured by the lambda expression by value:

```cpp
int divisor = 23;

std::cout
    << "Count (lambda explicitly capture by value): "
    << std::count_if(data.begin(), data.end(), [divisor](int v){ return 0 == v%divisor; })
    << " (divisor: " << divisor << ")"
    << std::endl;
```

Use `=` to implicitly capture by value:

```cpp
std::cout
    << "Count (lambda implicitly capture by value): "
    << std::count_if(data.begin(), data.end(), [=](int v){ return 0 == v%divisor; })
    << " (divisor: " << divisor << ")"
    << std::endl;
```

Use `&` to capture by reference:

```cpp
std::cout
    << "Count (lambda explicitly capture by reference): "
    << std::count_if(data.begin(), data.end(), [&divisor](int v){ divisor = 10; return 0 == v%divisor; })
    << " (divisor: " << divisor << ")"
    << std::endl;
```

`&` can also be put standalone in `[]` to indicate that the default capture is by reference.

The execution results:

In [ ]:
!make -C 05_lambda clean ; make -C 05_lambda 03_closure
!05_lambda/03_closure

## Comments on functional style

The lambda expression and closure allow functional style of programming.  As shown in the `std::count_if` example, it is a convenient tool to reduce the lines of code.  It generally makes the code looks cleaner and easier to maintain.  That buys us time to do more important things or optimize performance hotspot.

But there are times that we cannot entrust the optimization to the compiler.  Lambda expressions are not the easist place to add intrinsics or assemblies.

When working on a container object equipped with proper iterator interface, I go with the functional style.  The lambda expression may help avoid expensive intermediate buffers.  It works well at least for the initial prototype.

# Exercises

1. Measure the performance between using an output vector and returning a new vector.

# References

1. [Copy elision](https://en.cppreference.com/w/cpp/language/copy_elision) at cppreference.com .
1. [Lambda expressions](https://en.cppreference.com/w/cpp/language/lambda) at cppreference.com .
1. [C++ Lambdas Under The Hood](https://web.mst.edu/~nmjxv3/articles/lambdas.html)